In [1]:
import torch
import numpy as np
import data
import tqdm
import models
from torchvision.transforms import v2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
EPOCHS = 50
BATCH_SIZE = 32
LR = 1e-6
OUT_CLASSES = 3

In [ ]:
def transform(image):
    """
    Data augmentation by random rotation, and random scaling between 0.8 to 1.2

    :param image: Image to be transformed
    :return: Transformed image
    """
    return v2.RandomAffine(degrees=(-180,180),scale=(0.8,1.2))(image)

In [3]:
train_dataset = data.LensingDataset('index.csv','../dataset/train/',
                                    transform=transform)
test_dataset = data.LensingDataset('index.csv','../dataset/val/',
                                   transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               shuffle=True,
                                               batch_size=BATCH_SIZE,
                                               num_workers=15)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              shuffle=True,
                                              batch_size=BATCH_SIZE,
                                              num_workers=15)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, image_channels=1, num_classes=3) -> None:
        super(Model, self).__init__()
        self.model = models.ResNet18(image_channels=image_channels,num_classes=num_classes).to(device)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
model = Model().to(device)
model.load_state_dict(torch.load('resnet_18.pt'))

loss_fn = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(params=model.parameters(),lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,'min',patience=3,factor=0.1)
history = {'val_loss':[], 'acc':[], 'loss':[]}
best_val_loss = np.inf
best_weights = None
for epoch in range(EPOCHS):
    losses = []
    accs = []
    model.train()
    for i,(x,y) in enumerate(tqdm.tqdm(train_dataloader, desc=f"Training epoch {epoch+1}")):
        (x, y) = (x.float().to(device), y.to(device))
        x = model(x)
        loss = loss_fn(x,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        label_indices = torch.argmax(y, 1)
        prediction_indices = torch.argmax(x, 1)
        acc = float(torch.sum(label_indices == prediction_indices))/float(len(label_indices))
        losses.append(float(loss))
        accs.append(float(acc))
    print("Training at epoch %d/%d: %.4f with acc=%.5f at LR=%.6f"%(epoch+1,EPOCHS,np.mean(losses),np.mean(accs),opt.param_groups[0]['lr']))
    history['loss'].append(np.mean(losses))
    train_loss = np.mean(losses)
    model.eval()
    losses = []
    accs = []
    for i,(x,y) in enumerate(tqdm.tqdm(test_dataloader, desc=f"Validation epoch {epoch}")):
        (x, y) = (x.float().to(device), y.to(device))
        x = model(x)
        val_loss = loss_fn(x,y)
        label_indices = torch.argmax(y, 1)
        prediction_indices = torch.argmax(x, 1)
        acc = float(torch.sum(label_indices == prediction_indices))/float(len(label_indices))
        losses.append(float(val_loss))
        accs.append(float(acc))
    if best_val_loss > np.mean(losses):
        best_val_loss = np.mean(losses)
        best_weights = model.state_dict()
    history['val_loss'].append(np.mean(losses))
    history['acc'].append(np.mean(accs))
    print(f"Validation at epoch {epoch+1}/{EPOCHS}: {np.mean(losses)} with acc={np.mean(accs)} at LR={opt.param_groups[0]['lr']}")
    scheduler.step(np.mean(losses))
    if opt.param_groups[0]['lr'] < 1e-9:
        print("Breaking as learning rate is too small")
        break



Training epoch 1: 100%|██████████| 938/938 [01:55<00:00,  8.11it/s]


Training at epoch 1/1000: 0.2883 with acc=0.88999 at LR=0.000001


Validation epoch 0: 100%|██████████| 235/235 [00:11<00:00, 21.10it/s]


Validation at epoch 1/1000: 0.403625534649225 with acc=0.8672429078014183 at LR=1e-06


Training epoch 2: 100%|██████████| 938/938 [01:53<00:00,  8.28it/s]


...

Training epoch 51: 100%|██████████| 938/938 [01:53<00:00,  8.24it/s]


Training at epoch 51/1000: 0.2504 with acc=0.90605 at LR=0.000000


Validation epoch 50: 100%|██████████| 235/235 [00:11<00:00, 21.15it/s]


Validation at epoch 51/1000: 0.43362827031536305 with acc=0.8714095744680851 at LR=1e-08


Training epoch 52:  73%|███████▎  | 684/938 [01:23<00:30,  8.20it/s]


In [ ]:
model.load_state_dict(best_weights)
torch.save(model.state_dict(), '../trained_weights/resnet_18_fine.pt')